# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Output File (CSV)
output_data = "../Output Data File/output_data.csv"

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_wx = pd.read_csv(output_data)
city_wx.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,praia da vitoria,38.7333,-27.0667,62.01,94,40,12.73,PT,1615084287
1,1,samusu,-33.6500,115.3333,73.00,76,13,1.99,AU,1615084248
2,2,albany,42.6001,-73.9662,23.00,55,3,1.01,US,1615084201
3,3,avarua,-21.2078,-159.7750,78.80,94,75,14.97,CK,1615084339
4,4,hami,42.8000,93.4500,43.74,53,0,5.91,CN,1615084339


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [11]:
gmaps.configure(api_key=g_key)

locations = city_wx[["Lat","Lng"]]
humidity = city_wx["Humidity"]

heatmap = gmaps.figure(center = [0,0], zoom_level = 2.0)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max(humidity), point_radius = 2.8,)

heatmap.add_layer(heat_layer)

heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
city_temp = city_wx.loc[(city_wx["Max Temp"] < 80) & (city_wx["Max Temp"] >= 70)]
city_wx_best = city_temp.loc[(city_temp["Wind Speed"] < 10) & (city_temp["Cloudiness"] == 0)]
city_wx_best

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,79,tura,25.5198,90.2201,77.25,54,0,4.29,IN,1615084352
118,118,lazaro cardenas,17.9583,-102.2000,73.08,82,0,2.24,MX,1615084357
171,171,wattegama,6.7989,81.4808,77.00,100,0,3.44,LK,1615084366
178,178,tecoanapa,16.5167,-98.7500,74.73,80,0,5.06,MX,1615084367
246,246,formosa,-26.1775,-58.1781,75.20,78,0,4.61,AR,1615084381
277,277,pilar,-9.5972,-35.9567,75.20,94,0,3.44,BR,1615084386
284,284,ipixuna,-1.7625,-48.8043,75.20,94,0,2.30,BR,1615084387
319,319,kidal,18.4411,1.4078,76.30,18,0,9.44,ML,1615084392
331,331,simao,22.7886,100.9748,76.37,20,0,3.94,CN,1615084394
341,341,pinotepa nacional,16.3167,-98.0167,70.92,81,0,2.57,MX,1615084394


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df = city_wx_best.copy()
hotel_df["Hotel Name"] = ""

In [25]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "hotel",
          "radius": 5000,
          "keyword":"hotel",
          "key": g_key}

for index,row in hotel_df.iterrows():
    params['location'] = f"{row['Lat']}, {row['Lng']} "
    response = requests.get(url, params=params).json()
    results = response["results"]
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except(KeyError, IndexError):
        pass

In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add marker layer ontop of heat map
hotels = gmaps.marker_layer(locations, info_box_content= hotel_info)
heatmap.add_layer(hotels)

# Display figure
heatmap

Figure(layout=FigureLayout(height='420px'))